In [2]:
import grpc

import Api_pb2 as api
import Api_pb2_grpc as api_grpc
import pandas as pd
# import route_guide_resources

    

Получение инфо о ТС

In [39]:

vehicles_request = api.GetVehiclesRequest(Subsystems=["kiutr"])

with grpc.insecure_channel(endpoint_address) as channel:

    print("------------Create Stub--------------")
    stub = api_grpc.APIStub(channel)
    print(stub)
    
    print("------------Get objects info-----------")
    vehicles = stub.GetVehicles(vehicles_request)
    print(objects_info_responce)


------------Create Stub--------------
------------Get objects info-----------


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "grpc: error while marshaling: proto: Marshal called with nil"
	debug_error_string = "{"created":"@1605545893.095081705","description":"Error received from peer ipv4:185.98.83.83:18082","file":"src/core/lib/surface/call.cc","file_line":1061,"grpc_message":"grpc: error while marshaling: proto: Marshal called with nil","grpc_status":13}"
>

Получение информации об объектах

In [37]:
timeFrom = pd.to_datetime("2020-09-13 21:00:00", format='%Y-%m-%d %H:%M:%S')
timeTo = pd.to_datetime("2020-09-14 21:00:00", format='%Y-%m-%d %H:%M:%S')

objects_info_request = api.ObjectsInfoRequest(Filter= api.DataFilter(
        DateFrom= 			94489479168,#int(timeFrom.timestamp()),
        DateTo= 			94789479168#int(timeTo.timestamp()),
        # Subsystem= 			"kiutr", # для мусоровозов - garbade, на тестовом стенде данные по garbage отсутствуют 
        # ExcludeDeviceCode= 	["10033473", "404957","500459"], # пример исключения уже обработанных блоков
        # DeviceCode=			["356850083601698"],  # дополнительные коды БНСО
        # StateNumber= 		["Н040РА195"],  # дополнительные госномера
    ))

In [38]:
endpoint_address = 'rnis-tm.t1-group.ru:18082'
with grpc.insecure_channel(endpoint_address) as channel:

    print("------------Create Stub--------------")
    stub = api_grpc.APIStub(channel)
    print(stub)
    
    print("------------Get objects info-----------")
    objects_info_responce = stub.GetObjectsInfo(objects_info_request)
    print(objects_info_responce)


------------Create Stub--------------
------------Get objects info-----------


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "grpc: error while marshaling: proto: Marshal called with nil"
	debug_error_string = "{"created":"@1605545275.778071972","description":"Error received from peer ipv4:185.98.83.83:18082","file":"src/core/lib/surface/call.cc","file_line":1061,"grpc_message":"grpc: error while marshaling: proto: Marshal called with nil","grpc_status":13}"
>

получение событий по устройству за период

Подписка на поток

In [45]:
timeFrom = pd.to_datetime("2020-09-13 21:00:00", format='%Y-%m-%d %H:%M:%S')
timeTo = pd.to_datetime("2020-09-15 21:00:00", format='%Y-%m-%d %H:%M:%S')

rangeStreamRequest = api.ObjectsDataRangeRequest(
    Filter= api.DataFilter(
        DateFrom= 			int(timeFrom.timestamp()),
        DateTo= 			int(timeTo.timestamp()),
        Subsystem= 			"kiutr", # для мусоровозов - garbade, на тестовом стенде данные по garbage отсутствуют 
        # ExcludeDeviceCode= 	["10033473", "404957","500459"], # пример исключения уже обработанных блоков
        # DeviceCode=			["10033473","404957"],  # дополнительные коды БНСО
        # StateNumber= 		["Н040РА195"],  # дополнительные госномера
    ),
    Fields = api.FieldsToggle(
        Position=True # запрашивает только навигационную информацию
    )
)



In [10]:
endpoint_address = 'rnis-tm.t1-group.ru:18082'
# endpoint_address = 'rnis-tm.t1-group.xu:000000'

# localhost = 'localhost:50051'
max_records = 250
with grpc.insecure_channel(endpoint_address) as channel:

    print("------------Create Stub--------------")
    stub = api_grpc.APIStub(channel)
    print(stub)
    
    print("------------Get DataStream-----------")
    stream = stub.GetObjectsDataStream(rangeStreamRequest)
    # stream = script.get_data_stream(stub)
    print(stream)

    print("------ Retreive data from stream-----")
    i = 0
    for data_point in stream:
        i += 1
        if i == max_records:
            break


        device_time = data_point.DeviceTime
        device_code = data_point.DeviceCode
        gps_data = data_point.Position
        accelerations = data_point.Accelerations

        longitude = gps_data.Longitude
        latitude = gps_data.Latitude
        altitude = gps_data.Altitude
        course = gps_data.Course
        speed = gps_data.Speed
        valid = gps_data.Valid
        hdop = gps_data.HDOP

        print('datapoint ', i)
        print('device_code: ', device_code)
        print('device_time: ', device_time)
        print('longitude: ', longitude)
        print('latitude: ', latitude)
        print('altitude: ', altitude)
        print('course: ', course)
        print('speed: ', speed)
        print('accelerations: ', accelerations)
        print('valid: ', valid)
        print('hdop: ', hdop)
        print()

        # add to dataframe
        record = {
            'device_code': device_code,
            # 'state_number': object_data.StateNumber,
            'device_time': device_time,            
            # 'state': data_point.ObjectState,
            # 'received_time': data_point.ReceivedTime,
            'accelerations': accelerations,
            # 'fuel_spent': data_point.FuelSpent,
            'longitude': longitude,
            'latitude': latitude,
            'altitude': altitude,
            'course': course,
            # 'satellites': satellites,
            'speed': speed,
            'valid': valid,
            'hdop': hdop,
        }

        records_df = records_df.append(record, ignore_index=True)

53607948834
altitude:  0.0
course:  129
speed:  0
accelerations:  
valid:  False
hdop:  0

datapoint  2
device_code:  46112611
device_time:  1605540467
longitude:  43.47680033265539
latitude:  56.23896540520688
altitude:  0.0
course:  67
speed:  20
accelerations:  
valid:  False
hdop:  0

datapoint  3
device_code:  46112611
device_time:  1605540468
longitude:  43.476902256597974
latitude:  56.23898149846098
altitude:  0.0
course:  78
speed:  21
accelerations:  
valid:  False
hdop:  0

datapoint  4
device_code:  46112611
device_time:  1605540469
longitude:  43.47701759158564
latitude:  56.23899759171507
altitude:  0.0
course:  83
speed:  22
accelerations:  
valid:  False
hdop:  0

datapoint  5
device_code:  46112611
device_time:  1605540489
longitude:  43.47887368022438
latitude:  56.23911024449372
altitude:  0.0
course:  80
speed:  0
accelerations:  
valid:  False
hdop:  0

datapoint  6
device_code:  40931529
device_time:  1605540409
longitude:  43.389019678204555
latitude:  56.2268901

Получение данных по фильтру

In [3]:
timeFrom = pd.to_datetime("2020-09-13 21:00:00", format='%Y-%m-%d %H:%M:%S')
timeTo = pd.to_datetime("2020-09-15 21:00:00", format='%Y-%m-%d %H:%M:%S')

rangeStreamRequest = api.ObjectsDataRangeRequest(
    Filter= api.DataFilter(
        # DateFrom= 			int(str(timeFrom.timestamp())[:-2], 16),
        # DateTo= 			int(str(timeTo.timestamp())[:-2], 16),
        DateFrom= 			int(timeFrom.timestamp()),
        DateTo= 			int(timeTo.timestamp()),
        Subsystem= 			"kiutr", # для мусоровозов - garbade, на тестовом стенде данные по garbage отсутствуют 
        # ExcludeDeviceCode= 	["10033473", "404957","500459"], # пример исключения уже обработанных блоков
        # DeviceCode=			["10033473","404957"],  # дополнительные коды БНСО
        # StateNumber= 		["Н040РА195"],  # дополнительные госномера
    ),
    Fields = api.FieldsToggle(
        Position=True # запрашивает только навигационную информацию
    )
)



In [5]:
endpoint_address = 'rnis-tm.t1-group.ru:18082'
# endpoint_address = 'rnis-tm.t1-group.xu:000000'

# localhost = 'localhost:50051'
with grpc.insecure_channel(endpoint_address) as channel:

    print("------------Create Stub--------------")
    stub = api_grpc.APIStub(channel)
    print(stub)
    
    print("------------Get DataStream-----------")
    stream = stub.GetObjectsDataRangeAsStream(rangeStreamRequest)
    # stream = script.get_data_stream(stub)
    print(stream)

    print("------ Retreive data from stream-----")
    i = 0
    for object_data in stream:
        i += 1
        if i == 5:
            break

        device_code = object_data.DeviceCode
        # state_number = object_data.StateNumber
        data_point = object_data.DataPoint

        device_time = data_point.DeviceTime
        gps_data = data_point.Position
        state = data_point.ObjectState
        # received_time = data_point.ReceivedTime
        accelerations = data_point.Accelerations
        # fuel_spent = data_point.FuelSpent

        longitude = gps_data.Longitude
        latitude = gps_data.Latitude
        altitude = gps_data.Altitude
        course = gps_data.Course
        # satellites = gps_data.Satellites
        speed = gps_data.Speed
        valid = gps_data.Valid
        hdop = gps_data.HDOP

        print('datapoint ', i)
        print('device_code: ', device_code)
        print('device_time: ', device_time)
        print('state: ', state)
        print('longitude: ', longitude)
        print('latitude: ', latitude)
        print('altitude: ', altitude)
        print('course: ', course)
        print('speed: ', speed)
        print('accelerations: ', accelerations)
        print('valid: ', valid)
        print('hdop: ', hdop)
        print()

------------Create Stub--------------
------------Get DataStream-----------
<_MultiThreadedRendezvous object>
------ Retreive data from stream-----


In [13]:
# with grpc.insecure_channel(endpoint_address) as channel:

#     print("------------Create Stub--------------")
#     stub = api_grpc.APIStub(channel)
#     print(stub)
    
#     print("------------Get DataStream-----------")
#     stream = stub.GetObjectsDataRangeAsStream(rangeStreamRequest)
#     # stream = script.get_data_stream(stub)
#     print(stream)

In [14]:
print("------------Open Channel--------------")
channel = grpc.insecure_channel(endpoint_address)
channel




------------Open Channel--------------


In [15]:
print("------------Create Stub--------------")
stub = api_grpc.APIStub(channel)
stub



------------Create Stub--------------


In [16]:
print("------------Get DataStream-----------")
stream = stub.GetObjectsDataRangeAsStream(rangeStreamRequest)
# stream = script.get_data_stream(stub)
stream

------------Get DataStream-----------


<_MultiThreadedRendezvous object>

In [17]:
for _ in stream:
    print(_)

_

<_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.OK
	details = ""
>

In [11]:
print("------------Close Channel--------------")
channel.close()

------------Close Channel--------------


In [16]:
# records_df = pd.DataFrame(columns=['device_code', 
#                             'state_number',
#                             'course',
#                             'device_time',
#                             'state',
#                             'longitude',
#                             'latitude',
#                             'altitude',
#                             'speed', 
#                             'accelerations',
#                             'fuel_spent',
#                             'satellites'
#                             'valid',
#                             'hdop'])

    
# with grpc.insecure_channel(endpoint_address) as channel:
#     print("------------Create Stub--------------")
#     stub = api_grpc.APIStub(channel)

#     print("------------Get DataStream-----------")
#     stream = stub.GetObjectsDataRangeAsStream(rangeStreamRequest)

#     print("------ Retreive data from stream-----")

#     i = 0
#     for object_data in stream:
#         i += 1
#         if i == max_records:
#             break

#         data_point = object_data.DataPoint
#         gps_data = data_point.Position
#         record = {
#             'device_code': object_data.DeviceCode,
#             'state_number': object_data.StateNumber,
#             'device_time': data_point.DeviceTime,            
#             'state': data_point.ObjectState,
#             'received_time': data_point.ReceivedTime,
#             'accelerations': data_point.Accelerations,
#             'fuel_spent': data_point.FuelSpent,
#             'longitude': gps_data.Longitude,
#             'latitude': gps_data.Latitude,
#             'altitude': gps_data.Altitude,
#             'course': gps_data.Course,
#             'satellites': gps_data.Satellites,
#             'speed': gps_data.Speed,
#             'valid': gps_data.Valid,
#             'hdop': gps_data.HDOP,
#         }

------------Create Stub--------------
------------Get DataStream-----------
------ Retreive data from stream-----
